In [1]:
import tensorflow as tf
import cv2
from tensorflow.python.training import saver as tf_saver

In [2]:
from tensorflow.models.slim.datasets import custom_data, dataset_factory

In [3]:
slim = tf.contrib.slim

In [4]:
def preprocessing(image, size):
    image = tf.to_float(image)
    image = tf.image.resize_image_with_crop_or_pad(image, size, size)
    return image

In [8]:
def inference(images, num_classes=2, is_training=False, prediction_fn=slim.softmax, scope='bnw'):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu):
        #with tf.variable_scope(scope, 'bnw', [images, num_classes]):
        net = slim.conv2d(images, 64, [5, 5], scope='conv1')
        print("conv1 --> %s"%net.get_shape())
        net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool1')
        print("pool1 --> %s"%net.get_shape())
        net = slim.conv2d(images, 64, [5, 5], scope='conv2')
        print("conv2 --> %s"%net.get_shape())
        net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool2')
        print("pool2 --> %s"%net.get_shape())
        net = slim.fully_connected(net, 192, scope='fc1')
        print("fc1 --> %s"%net.get_shape())
        net = slim.fully_connected(net, 2, scope='fc2')
        print("fc2 --> %s"%net.get_shape())
        net = slim.max_pool2d(net, [16, 16], scope='pool3')
        print("pool3 --> %s"%net.get_shape())
        logits = tf.squeeze(net, [1, 2], name='logits')
        print("logits --> %s"%net.get_shape())
    return logits


In [9]:
with tf.Graph().as_default() as g:
    tf_global_step = slim.get_or_create_global_step()
    img_data = tf.gfile.FastGFile("/home/siftr/siftr-images/1.jpg").read()
    img_data = tf.cast(tf.image.decode_jpeg(img_data, channels=3), tf.float32)
    img_data = tf.image.resize_image_with_crop_or_pad(img_data, 32, 32)
    img_data = tf.expand_dims(img_data, [0])
    print(img_data)

    logits = inference(img_data, is_training=False)
    predictions = tf.argmax(logits)
    print(predictions)

    sv = tf.train.Supervisor(logdir="/home/siftr/apus/dl/")
    with sv.managed_session() as sess:
        predict, log = sess.run([predictions, logits])
        print predict, log

Tensor("ExpandDims:0", shape=(1, 32, 32, 3), dtype=float32)
conv1 --> (1, 32, 32, 64)
pool1 --> (1, 16, 16, 64)
conv2 --> (1, 32, 32, 64)
pool2 --> (1, 16, 16, 64)
fc1 --> (1, 16, 16, 192)
fc2 --> (1, 16, 16, 2)
pool3 --> (1, 1, 1, 2)
logits --> (1, 1, 1, 2)
Tensor("ArgMax:0", shape=(2,), dtype=int64)
INFO:tensorflow:Restoring parameters from /home/siftr/apus/dl/model.ckpt-500
INFO:tensorflow:global_step/sec: 0
[0 0] [[ 0.  0.]]


In [10]:
##############################################

In [5]:
# def inference(images, num_classes=2, is_training=True, prediction_fn=slim.softmax, scope='bnw'):
#     with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu):
#         with tf.variable_scope(scope, 'bnw_var', [images, num_classes]):
#             net = slim.conv2d(images, 64, [5, 5], scope='conv1')
#             print("conv1 --> %s"%net.get_shape())
#             net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool1')
#             print("pool1 --> %s"%net.get_shape())
#             net = slim.conv2d(images, 64, [5, 5], scope='conv2')
#             print("conv2 --> %s"%net.get_shape())
#             net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool2')
#             print("pool2 --> %s"%net.get_shape())
#             net = slim.fully_connected(net, 192, scope='fc1')
#             print("fc1 --> %s"%net.get_shape())
#             net = slim.fully_connected(net, 2, scope='fc2')
#             print("fc2 --> %s"%net.get_shape())
#             net = slim.max_pool2d(net, [16, 16], scope='pool3')
#             print("pool3 --> %s"%net.get_shape())
#             logits = tf.squeeze(net, [1, 2], name='logits')
#             print("logits --> %s"%net.get_shape())
#     return logits

In [6]:
# #with tf.Graph().as_default() as g:
# #global_step = slim.create_global_step()
# image = cv2.imread("/home/siftr/siftr-images/38.jpg")
# print image.shape
# image = preprocessing(image, 32)
# print image.get_shape()
# image = tf.train.batch([image], batch_size=1, capacity=5)
# print image
# batch_queue = slim.prefetch_queue.prefetch_queue([image], capacity=2 * 1)
# inf_logit = inference(image, is_training=False)
# predict = tf.argmax(inf_logit, 1)

# saver = tf_saver.Saver()
# new_graph = tf.Graph()
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     new_saver = tf.train.import_meta_graph("/home/siftr/BNW/dl/model.ckpt-100.meta")
#     new_saver.restore(sess, '/home/siftr/BNW/dl/model.ckpt-100')    
#     print 1
#     p = sess.run(predict)    
#     print p